In [ ]:
import requests
from io import BytesIO
from PIL import Image
import os

from tqdm.auto import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

from ImageDataset import ImageDataset

In [ ]:
metadata = pd.read_csv('data/train.csv')
transform = T.Compose([
    T.CenterCrop((400, 400)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = ImageDataset(metadata, transform=transform)
loader = DataLoader(
    dataset,
    batch_size=1
)


In [ ]:
for batch in tqdm(loader):
    pass